In [1]:
# imports necesarios
from sklearn.datasets import fetch_openml
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier


### MNIST con GradientBoosting
En esta segunda sesión práctica vamos a entrenar clasificadores basados en GradientBoosting.
Claramente estos clasificadores son más costosos de entrenar.


In [2]:
mnist = fetch_openml("mnist_784")

print(mnist.data.shape)
print(mnist.target.shape)

data = mnist.data
targets = mnist.target 

targets=targets.to_numpy()
targets=np.int8(targets)

data=data.to_numpy()
data=np.float32(data)

(70000, 784)
(70000,)


### Partición de los datos

Vamos a partir los datos en tres conjuntos: training, validation y test. Con un 80%, 10% y 10% respectivamente. 

Emplearemos el conjunto de training para aprender los parámetros del modelos, el conjunto de validation para escoger los mejores hiperparámetros. Finalmente reportaremos el resultado final sobre el conjunto de test.

### **Ejercicio 1**    

Realiza la partición de los datos en las particiones definidas (80%,10%,10%)

In [3]:
# Solución

x_train, x_test, y_train, y_test  = train_test_split(data, targets, test_size=0.1, random_state=1)

part=(7000.0)/(70000.0-7000)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=part, random_state=1) 

print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(56000, 784)
(7000, 784)
(7000, 784)


### **Ejercicio 2**   

Prueba un clasificador GradientBoosting

In [ ]:
## Solución
gb = GradientBoostingClassifier()
gb.fit(x_train,y_train)
    
ypred = gb.predict(x_test)
acc = accuracy_score(y_test, ypred)
print(acc)

Podrás comprobar como es un clasificador mucho más lento. 

### **Ejercicio 3**   

Para reducir el coste computacional se propone crear un pipeline donde se reduzca el número de características mediante PCA. En concreto el número de componentes (dimensiones) a las que reducimos con PCA es un hyperparámetro que tendrás que estimar con el conjunto de validación.

In [4]:
## Solución
from sklearn.pipeline import make_pipeline
from sklearn.decomposition import PCA

for n in [2,4,8,16,32]:
    clf = make_pipeline(PCA(n_components=n),GradientBoostingClassifier())
    print(n)  
    clf.fit(x_train, y_train)
    ypred = clf.predict(x_val)
    acc = accuracy_score(y_val, ypred)
    print(acc)  


2
0.47585714285714287
4
0.6548571428571428
8
0.8685714285714285
16
0.9151428571428571
32
0.927


De entre los diferentes parámeros que tiene el clasificador GradientBoosting de sklearn, cabría destacar:

**learning_rate**

**n_estimators**

**min_samples_split**

**max_features** 


Para más información leer la documentación en sklearn.

Alguno de estos parámetros influyen considerablemente en la velocidad de optimización. Por ejemplo **max_features** y **min_samples_split** entre otros.



### **Ejercicio 4**   

Se propone variar alguno de estos parámetros para ver si se obtiene una similar tasa de acierto pero con mejor velocidad.


In [5]:
# Solución en celdas separadas para medir tiempo

# Coste original : 3m 30 s
# Tasa de acierto: 0.927
clf = make_pipeline(PCA(n_components=32),
        GradientBoostingClassifier())
clf.fit(x_train, y_train)

ypred = clf.predict(x_val)
acc = accuracy_score(y_val, ypred)
print(n,acc)  

32 0.9288571428571428


In [6]:
# Solución, truco max_features=sqrt(dim) approx.
# Coste mejorado : 56 s casi 4x
# Tasa de acierto: 0.924 frete a 0.927
clf = make_pipeline(PCA(n_components=32),
        GradientBoostingClassifier(max_features=6, n_estimators=100, min_samples_split=10))
clf.fit(x_train, y_train)

ypred = clf.predict(x_val)
acc = accuracy_score(y_val, ypred)
print(n,acc)  

32 0.9258571428571428


Finalmente sklearn propone otro tipo de algoritmo de GradientBoosting que soporta paralelismo con OMP además de otras mejoras computacionales basadas en la discretización de las componentes mediante un histograma: el HistGradientBoostingClassifier. Su tiempo de ejecución es mucho mejor. Además se pueden obtener mejores resultados.



### **Ejercicio 5**   

Pruébalo y compara los tiempos de ejecución.

In [7]:
# Solución
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

clf = make_pipeline(PCA(n_components=32),
        HistGradientBoostingClassifier())
clf.fit(x_train, y_train)

ypred = clf.predict(x_val)
acc = accuracy_score(y_val, ypred)
print(n,acc) 

/home/ajuan/.local/lib/python3.10/site-packages/sklearn/experimental/enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


32 0.9671428571428572


### **Ejercicio 6**   

Prueba los parámetros del HistGradientBoostingClassifier que mejoren la tasa de acierto. En cualquier caso la selección de estos parámetros debe seguir el protocolo de experimentación antes expuesto. Esto es, escoger el mejor parámetro con datos de validación y reportar resultados con los datos de test.